[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PPatty666/Germany_Flood_Study/blob/main/Visuals/visual_scripts/Cleaning_up_Germany_Land_2020.ipynb)

In [ ]:
## Install required libraries
%%capture

!pip install fiona
!pip install folium mapclassify
!pip install ipyleaflet
!pip install -U kaleido
!pip install brokenaxes

In [ ]:
# Make a note
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Import libraries that will be possibly needed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
from brokenaxes import brokenaxes
import matplotlib.ticker as mticker
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

from ipyleaflet import Map, GeoJSON, Popup
from ipywidgets import HTML
import json
import folium
from folium import IFrame

import math
from numpy.random import default_rng
from scipy.stats import gamma
import arviz as az
import pymc as pm
import xarray as xr

import geopandas as gpd
import fiona
from shapely.geometry import MultiLineString, LineString
from shapely.geometry import Polygon, MultiPolygon

import pickle

import warnings
warnings.filterwarnings('ignore')

## Preprocessing for the entire Land dataset: **PLEASE update the file path to read in the raw file**

In [ ]:
!unzip "/content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/Land_all_return_periods.zip"

In [ ]:
file_paths = [
    '/content/Germany_Land_2020_5.xlsx',
    '/content/Germany_Land_2020_10.xlsx',
    '/content/Germany_Land_2020_20.xlsx',
    '/content/Germany_Land_2020_50.xlsx',
    '/content/Germany_Land_2020_100.xlsx',
    '/content/Germany_Land_2020_200.xlsx',
    '/content/Germany_Land_2020_500.xlsx',
    '/content/Germany_Land_2020_1000.xlsx'
]

dataframes = []
for path in file_paths:
    df = pd.read_excel(path, sheet_name='Land', skiprows=1)
    df = df.round(6)
    dataframes.append(df)

# Unpack into variables
(Land_2020_5_raw, Land_2020_10_raw, Land_2020_20_raw, Land_2020_50_raw,
 Land_2020_100_raw, Land_2020_200_raw, Land_2020_500_raw, Land_2020_1000_raw) = dataframes

In [ ]:
#Inspect column names from the raw dataset
Land_2020_100_raw.columns

In [ ]:
for df in [Land_2020_5_raw, Land_2020_10_raw, Land_2020_20_raw, Land_2020_50_raw,\
           Land_2020_100_raw, Land_2020_200_raw, Land_2020_500_raw, Land_2020_1000_raw]:
  df.rename(columns={'Unnamed: 0': 'Land'}, inplace=True)

In [ ]:
Land_2020_100_raw.Land

In [ ]:
## Create correct column names flood types + time
# Date range: every 6 months
dates = pd.date_range(start='2016-07-01', end='2025-01-01', freq='6MS')

# Prefixes and suffixes
prefixes = ['CU', 'FU', 'PD', 'MAX']
suffixes = ['P0', 'lt0.15', 'lt0.5', 'lt1.5', 'gt1.5']

# Correct order: by prefix, then suffix, then date
all_columns = [
    f"{prefix}-{date.strftime('%b-%y')}-{suffix}"
    for prefix in prefixes
    for suffix in suffixes
    for date in dates
]

In [ ]:
all_columns

In [ ]:
# Check if number of replacement columns matches the shape
for df in [Land_2020_5_raw, Land_2020_10_raw, Land_2020_20_raw, Land_2020_50_raw,\
           Land_2020_100_raw, Land_2020_200_raw, Land_2020_500_raw, Land_2020_1000_raw]:
    if len(all_columns) == df.shape[1] - 1:
      df.columns = [df.columns[0]] + all_columns
    else:
      raise ValueError("Length of generated column names does not match number of columns (excluding the first one).")


In [ ]:
Land_2020_100_raw.columns

## Coastal flooding undefended

### Data Preparation

In [ ]:
# Select columns that start with 'CU'
cu_columns = ['Land'] + [col for col in all_columns if col.startswith('CU')]

Land_2020_5_CU = Land_2020_5_raw[cu_columns]
Land_2020_10_CU = Land_2020_10_raw[cu_columns]
Land_2020_20_CU = Land_2020_20_raw[cu_columns]
Land_2020_50_CU = Land_2020_50_raw[cu_columns]
Land_2020_100_CU = Land_2020_100_raw[cu_columns]
Land_2020_200_CU = Land_2020_200_raw[cu_columns]
Land_2020_500_CU = Land_2020_500_raw[cu_columns]
Land_2020_1000_CU = Land_2020_1000_raw[cu_columns]

In [ ]:
Land_2020_5_CU.Land.unique()

In [ ]:
Land_2020_5_CU_long = Land_2020_5_CU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='CU_area')
Land_2020_10_CU_long = Land_2020_10_CU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='CU_area')
Land_2020_20_CU_long = Land_2020_20_CU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='CU_area')
Land_2020_50_CU_long = Land_2020_50_CU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='CU_area')
Land_2020_100_CU_long = Land_2020_100_CU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='CU_area')
Land_2020_200_CU_long = Land_2020_200_CU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='CU_area')
Land_2020_500_CU_long = Land_2020_500_CU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='CU_area')
Land_2020_1000_CU_long = Land_2020_1000_CU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='CU_area')

In [ ]:
Land_2020_5_CU_long

In [ ]:
for df in [Land_2020_5_CU_long, Land_2020_10_CU_long, Land_2020_20_CU_long, Land_2020_50_CU_long,\
           Land_2020_100_CU_long, Land_2020_200_CU_long, Land_2020_500_CU_long, Land_2020_1000_CU_long]:
    # Split the 'month-year-depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(CU)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Optional: drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)

In [ ]:
Land_2020_5_CU_settle = Land_2020_5_CU_long.groupby(['Land', 'month_year']).sum('CU_area').reset_index()
Land_2020_10_CU_settle = Land_2020_10_CU_long.groupby(['Land', 'month_year']).sum('CU_area').reset_index()
Land_2020_20_CU_settle = Land_2020_20_CU_long.groupby(['Land', 'month_year']).sum('CU_area').reset_index()
Land_2020_50_CU_settle = Land_2020_50_CU_long.groupby(['Land', 'month_year']).sum('CU_area').reset_index()
Land_2020_100_CU_settle = Land_2020_100_CU_long.groupby(['Land', 'month_year']).sum('CU_area').reset_index()
Land_2020_200_CU_settle = Land_2020_200_CU_long.groupby(['Land', 'month_year']).sum('CU_area').reset_index()
Land_2020_500_CU_settle = Land_2020_500_CU_long.groupby(['Land', 'month_year']).sum('CU_area').reset_index()
Land_2020_1000_CU_settle = Land_2020_1000_CU_long.groupby(['Land', 'month_year']).sum('CU_area').reset_index()

In [ ]:
Land_2020_5_CU_settle

In [ ]:
for df in [Land_2020_5_CU_settle, Land_2020_10_CU_settle, Land_2020_20_CU_settle, Land_2020_50_CU_settle,\
           Land_2020_100_CU_settle, Land_2020_200_CU_settle, Land_2020_500_CU_settle, Land_2020_1000_CU_settle]:
    df.columns = ['Land', 'month_year', 'settle_area']

In [ ]:
Land_2020_5_CU_df = pd.merge(Land_2020_5_CU_long, Land_2020_5_CU_settle, on=['Land', 'month_year'], how='left')
Land_2020_10_CU_df = pd.merge(Land_2020_10_CU_long, Land_2020_10_CU_settle, on=['Land', 'month_year'], how='left')
Land_2020_20_CU_df = pd.merge(Land_2020_20_CU_long, Land_2020_20_CU_settle, on=['Land', 'month_year'], how='left')
Land_2020_50_CU_df = pd.merge(Land_2020_50_CU_long, Land_2020_50_CU_settle, on=['Land', 'month_year'], how='left')
Land_2020_100_CU_df = pd.merge(Land_2020_100_CU_long, Land_2020_100_CU_settle, on=['Land', 'month_year'], how='left')
Land_2020_200_CU_df = pd.merge(Land_2020_200_CU_long, Land_2020_200_CU_settle, on=['Land', 'month_year'], how='left')
Land_2020_500_CU_df = pd.merge(Land_2020_500_CU_long, Land_2020_500_CU_settle, on=['Land', 'month_year'], how='left')
Land_2020_1000_CU_df = pd.merge(Land_2020_1000_CU_long, Land_2020_1000_CU_settle, on=['Land', 'month_year'], how='left')

In [ ]:
Land_2020_5_CU_df.head()

In [ ]:
for df in [Land_2020_5_CU_df, Land_2020_10_CU_df, Land_2020_20_CU_df, Land_2020_50_CU_df,\
           Land_2020_100_CU_df, Land_2020_200_CU_df, Land_2020_500_CU_df, Land_2020_1000_CU_df]:
    df['depth_cat_area_perc'] = (df.CU_area / df.settle_area)*100

In [ ]:
Land_2020_5_CU_baseline = Land_2020_5_CU_df[Land_2020_5_CU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'CU_area']]
Land_2020_10_CU_baseline = Land_2020_10_CU_df[Land_2020_10_CU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'CU_area']]
Land_2020_20_CU_baseline = Land_2020_20_CU_df[Land_2020_20_CU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'CU_area']]
Land_2020_50_CU_baseline = Land_2020_50_CU_df[Land_2020_50_CU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'CU_area']]
Land_2020_100_CU_baseline = Land_2020_100_CU_df[Land_2020_100_CU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'CU_area']]
Land_2020_200_CU_baseline = Land_2020_200_CU_df[Land_2020_200_CU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'CU_area']]
Land_2020_500_CU_baseline = Land_2020_500_CU_df[Land_2020_500_CU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'CU_area']]
Land_2020_1000_CU_baseline = Land_2020_1000_CU_df[Land_2020_1000_CU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'CU_area']]

In [ ]:
Land_2020_20_CU_baseline.head()

In [ ]:
Land_2020_5_CU_baseline = Land_2020_5_CU_baseline.rename(columns={'CU_area': 'CU_area_Jul16'})
Land_2020_10_CU_baseline = Land_2020_10_CU_baseline.rename(columns={'CU_area': 'CU_area_Jul16'})
Land_2020_20_CU_baseline = Land_2020_20_CU_baseline.rename(columns ={'CU_area': 'CU_area_Jul16'})
Land_2020_50_CU_baseline = Land_2020_50_CU_baseline.rename(columns={'CU_area': 'CU_area_Jul16'})
Land_2020_100_CU_baseline = Land_2020_100_CU_baseline.rename(columns={'CU_area': 'CU_area_Jul16'})
Land_2020_200_CU_baseline = Land_2020_200_CU_baseline.rename(columns={'CU_area': 'CU_area_Jul16'})
Land_2020_500_CU_baseline = Land_2020_500_CU_baseline.rename(columns={'CU_area': 'CU_area_Jul16'})
Land_2020_1000_CU_baseline = Land_2020_1000_CU_baseline.rename(columns={'CU_area': 'CU_area_Jul16'})

In [ ]:
for df in [Land_2020_5_CU_baseline, Land_2020_10_CU_baseline, Land_2020_20_CU_baseline, Land_2020_50_CU_baseline,\
           Land_2020_100_CU_baseline, Land_2020_200_CU_baseline, Land_2020_500_CU_baseline, Land_2020_1000_CU_baseline]:
    df['CU_area_Jul16'] = df['CU_area_Jul16'].replace(0, 1e-6)

In [ ]:
Land_2020_5_CU_baseline.head()

In [ ]:
# Merge baseline back to original df
Land_2020_5_CU_df = Land_2020_5_CU_df.merge(Land_2020_5_CU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_10_CU_df = Land_2020_10_CU_df.merge(Land_2020_10_CU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_20_CU_df = Land_2020_20_CU_df.merge(Land_2020_20_CU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_50_CU_df = Land_2020_50_CU_df.merge(Land_2020_50_CU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_100_CU_df = Land_2020_100_CU_df.merge(Land_2020_100_CU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_200_CU_df = Land_2020_200_CU_df.merge(Land_2020_200_CU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_500_CU_df = Land_2020_500_CU_df.merge(Land_2020_500_CU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_1000_CU_df = Land_2020_1000_CU_df.merge(Land_2020_1000_CU_baseline, on=['Land', 'depth_cat'], how='left')

In [ ]:
for df in [Land_2020_5_CU_df, Land_2020_10_CU_df, Land_2020_20_CU_df, Land_2020_50_CU_df,\
           Land_2020_100_CU_df, Land_2020_200_CU_df, Land_2020_500_CU_df, Land_2020_1000_CU_df]:
    df['CU_area_non_zero'] = df.CU_area.replace(0, 1e-6)
    df['pct_growth'] = (df['CU_area_non_zero'] - df['CU_area_Jul16']) / df['CU_area_Jul16'] * 100

### Make an order for plotting

In [ ]:
def risk_level(value):
    if value == 'P0' or value == 'lt0.15':
        return 'low_risk'
    else:
        return 'high_risk'

In [ ]:
for df in [Land_2020_5_CU_df, Land_2020_10_CU_df, Land_2020_20_CU_df, Land_2020_50_CU_df,\
           Land_2020_100_CU_df, Land_2020_200_CU_df, Land_2020_500_CU_df, Land_2020_1000_CU_df]:
    df['risk_level'] = df.depth_cat.map(risk_level)

In [ ]:
Land_2020_5_CU_df.head()

In [ ]:
Land_2020_5_jan25_risky_CU = Land_2020_5_CU_df[(Land_2020_5_CU_df['month_year'] == 'Jan-25') & (Land_2020_5_CU_df['risk_level'] == 'high_risk')]
Land_2020_10_jan25_risky_CU = Land_2020_10_CU_df[(Land_2020_10_CU_df['month_year'] == 'Jan-25') & (Land_2020_10_CU_df['risk_level'] == 'high_risk')]
Land_2020_20_jan25_risky_CU = Land_2020_20_CU_df[(Land_2020_20_CU_df['month_year'] == 'Jan-25') & (Land_2020_20_CU_df['risk_level'] == 'high_risk')]
Land_2020_50_jan25_risky_CU = Land_2020_50_CU_df[(Land_2020_50_CU_df['month_year'] == 'Jan-25') & (Land_2020_50_CU_df['risk_level'] == 'high_risk')]
Land_2020_100_jan25_risky_CU = Land_2020_100_CU_df[(Land_2020_100_CU_df['month_year'] == 'Jan-25') & (Land_2020_100_CU_df['risk_level'] == 'high_risk')]
Land_2020_200_jan25_risky_CU = Land_2020_200_CU_df[(Land_2020_200_CU_df['month_year'] == 'Jan-25') & (Land_2020_200_CU_df['risk_level'] == 'high_risk')]
Land_2020_500_jan25_risky_CU = Land_2020_500_CU_df[(Land_2020_500_CU_df['month_year'] == 'Jan-25') & (Land_2020_500_CU_df['risk_level'] == 'high_risk')]
Land_2020_1000_jan25_risky_CU = Land_2020_1000_CU_df[(Land_2020_1000_CU_df['month_year'] == 'Jan-25') & (Land_2020_1000_CU_df['risk_level'] == 'high_risk')]

In [ ]:
# Gemeinde_2020_5_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Land_2020_100_jan25_risky_sum_CU = Land_2020_100_jan25_risky_CU.groupby(['Land', 'risk_level']).sum(['CU_area_non_zero', 'CU_area_Jul16']).reset_index()[['Land', 'risk_level', 'CU_area_non_zero', 'CU_area_Jul16']]

In [ ]:
Land_2020_100_jan25_risky_sum_CU['pct_growth_risky'] = \
 (Land_2020_100_jan25_risky_sum_CU.CU_area_non_zero - Land_2020_100_jan25_risky_sum_CU.CU_area_Jul16) / Land_2020_100_jan25_risky_sum_CU.CU_area_Jul16 * 100

In [ ]:
Land_2020_100_jan25_risky_sum_CU = Land_2020_100_jan25_risky_sum_CU[Land_2020_100_jan25_risky_sum_CU.Land != "Germany"]

In [ ]:
Land_2020_100_jan25_risky_sum_CU.sort_values(by='pct_growth_risky', ascending=False).Land.tolist()

In [ ]:
cu_plot_order = Land_2020_100_jan25_risky_sum_CU.sort_values(by='pct_growth_risky', ascending=False).Land.tolist()

## Fluvial flooding undefended

### Data Preparation

In [ ]:
# Select columns that start with 'CU' or are exactly 'Land'
fu_columns = ['Land'] + [col for col in all_columns if col.startswith('FU')]

Land_2020_5_FU = Land_2020_5_raw[fu_columns]
Land_2020_10_FU = Land_2020_10_raw[fu_columns]
Land_2020_20_FU = Land_2020_20_raw[fu_columns]
Land_2020_50_FU = Land_2020_50_raw[fu_columns]
Land_2020_100_FU = Land_2020_100_raw[fu_columns]
Land_2020_200_FU = Land_2020_200_raw[fu_columns]
Land_2020_500_FU = Land_2020_500_raw[fu_columns]
Land_2020_1000_FU = Land_2020_1000_raw[fu_columns]

In [ ]:
Land_2020_5_CU.Land.unique()

In [ ]:
Land_2020_5_FU_long = Land_2020_5_FU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='FU_area')
Land_2020_10_FU_long = Land_2020_10_FU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='FU_area')
Land_2020_20_FU_long = Land_2020_20_FU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='FU_area')
Land_2020_50_FU_long = Land_2020_50_FU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='FU_area')
Land_2020_100_FU_long = Land_2020_100_FU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='FU_area')
Land_2020_200_FU_long = Land_2020_200_FU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='FU_area')
Land_2020_500_FU_long = Land_2020_500_FU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='FU_area')
Land_2020_1000_FU_long = Land_2020_1000_FU.melt(id_vars=['Land'], var_name='month_year_depth', value_name='FU_area')

In [ ]:
Land_2020_5_FU_long.head()

In [ ]:
for df in [Land_2020_5_FU_long, Land_2020_10_FU_long, Land_2020_20_FU_long, Land_2020_50_FU_long,\
           Land_2020_100_FU_long, Land_2020_200_FU_long, Land_2020_500_FU_long, Land_2020_1000_FU_long]:
    # Split the 'month-year-depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(FU)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Optional: drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)

In [ ]:
Land_2020_5_FU_long.head()

In [ ]:
Land_2020_5_FU_settle = Land_2020_5_FU_long.groupby(['Land', 'month_year']).sum('FU_area').reset_index()
Land_2020_10_FU_settle = Land_2020_10_FU_long.groupby(['Land', 'month_year']).sum('FU_area').reset_index()
Land_2020_20_FU_settle = Land_2020_20_FU_long.groupby(['Land', 'month_year']).sum('FU_area').reset_index()
Land_2020_50_FU_settle = Land_2020_50_FU_long.groupby(['Land', 'month_year']).sum('FU_area').reset_index()
Land_2020_100_FU_settle = Land_2020_100_FU_long.groupby(['Land', 'month_year']).sum('FU_area').reset_index()
Land_2020_200_FU_settle = Land_2020_200_FU_long.groupby(['Land', 'month_year']).sum('FU_area').reset_index()
Land_2020_500_FU_settle = Land_2020_500_FU_long.groupby(['Land', 'month_year']).sum('FU_area').reset_index()
Land_2020_1000_FU_settle = Land_2020_1000_FU_long.groupby(['Land', 'month_year']).sum('FU_area').reset_index()

In [ ]:
for df in [Land_2020_5_FU_settle, Land_2020_10_FU_settle, Land_2020_20_FU_settle, Land_2020_50_FU_settle,\
           Land_2020_100_FU_settle, Land_2020_200_FU_settle, Land_2020_500_FU_settle, Land_2020_1000_FU_settle]:
    df.columns = ['Land', 'month_year', 'settle_area']

In [ ]:
# Choose the column to compare
column = 'settle_area'

# Compare values row by row
matches = Land_2020_10_FU_settle[column] == Land_2020_10_CU_settle[column]

# Compare using np.isclose with a tolerance
matches = np.isclose(Land_2020_5_FU_settle[column], Land_2020_5_CU_settle[column], atol=1e-5)

# Print mismatches
for i, match in enumerate(matches):
    if not match:
        val1 = Land_2020_5_FU_settle.at[i, column]
        val2 = Land_2020_5_CU_settle.at[i, column]
        print(f"Row {i}: Land_2020_5_FU_settle = {val1}, Land_2020_5_CU_settle = {val2}")

In [ ]:
Land_2020_5_FU_df = pd.merge(Land_2020_5_FU_long, Land_2020_5_FU_settle, on=['Land', 'month_year'], how='left')
Land_2020_10_FU_df = pd.merge(Land_2020_10_FU_long, Land_2020_10_FU_settle, on=['Land', 'month_year'], how='left')
Land_2020_20_FU_df = pd.merge(Land_2020_20_FU_long, Land_2020_20_FU_settle, on=['Land', 'month_year'], how='left')
Land_2020_50_FU_df = pd.merge(Land_2020_50_FU_long, Land_2020_50_FU_settle, on=['Land', 'month_year'], how='left')
Land_2020_100_FU_df = pd.merge(Land_2020_100_FU_long, Land_2020_100_FU_settle, on=['Land', 'month_year'], how='left')
Land_2020_200_FU_df = pd.merge(Land_2020_200_FU_long, Land_2020_200_FU_settle, on=['Land', 'month_year'], how='left')
Land_2020_500_FU_df = pd.merge(Land_2020_500_FU_long, Land_2020_500_FU_settle, on=['Land', 'month_year'], how='left')
Land_2020_1000_FU_df = pd.merge(Land_2020_1000_FU_long, Land_2020_1000_FU_settle, on=['Land', 'month_year'], how='left')

In [ ]:
Land_2020_5_FU_df.head()

In [ ]:
for df in [Land_2020_5_FU_df, Land_2020_10_FU_df, Land_2020_20_FU_df, Land_2020_50_FU_df,\
           Land_2020_100_FU_df, Land_2020_200_FU_df, Land_2020_500_FU_df, Land_2020_1000_FU_df]:
    df['depth_cat_area_perc'] = (df.FU_area / df.settle_area)*100

In [ ]:
Land_2020_5_FU_df.head(n=20)

In [ ]:
Land_2020_5_FU_baseline = Land_2020_5_FU_df[Land_2020_5_FU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'FU_area']]
Land_2020_10_FU_baseline = Land_2020_10_FU_df[Land_2020_10_FU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'FU_area']]
Land_2020_20_FU_baseline = Land_2020_20_FU_df[Land_2020_20_FU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'FU_area']]
Land_2020_50_FU_baseline = Land_2020_50_FU_df[Land_2020_50_FU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'FU_area']]
Land_2020_100_FU_baseline = Land_2020_100_FU_df[Land_2020_100_FU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'FU_area']]
Land_2020_200_FU_baseline = Land_2020_200_FU_df[Land_2020_200_FU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'FU_area']]
Land_2020_500_FU_baseline = Land_2020_500_FU_df[Land_2020_500_FU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'FU_area']]
Land_2020_1000_FU_baseline = Land_2020_1000_FU_df[Land_2020_1000_FU_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'FU_area']]

In [ ]:
Land_2020_20_FU_baseline

In [ ]:
Land_2020_5_FU_baseline = Land_2020_5_FU_baseline.rename(columns={'FU_area': 'FU_area_Jul16'})
Land_2020_10_FU_baseline = Land_2020_10_FU_baseline.rename(columns={'FU_area': 'FU_area_Jul16'})
Land_2020_20_FU_baseline = Land_2020_20_FU_baseline.rename(columns ={'FU_area': 'FU_area_Jul16'})
Land_2020_50_FU_baseline = Land_2020_50_FU_baseline.rename(columns={'FU_area': 'FU_area_Jul16'})
Land_2020_100_FU_baseline = Land_2020_100_FU_baseline.rename(columns={'FU_area': 'FU_area_Jul16'})
Land_2020_200_FU_baseline = Land_2020_200_FU_baseline.rename(columns={'FU_area': 'FU_area_Jul16'})
Land_2020_500_FU_baseline = Land_2020_500_FU_baseline.rename(columns={'FU_area': 'FU_area_Jul16'})
Land_2020_1000_FU_baseline = Land_2020_1000_FU_baseline.rename(columns={'FU_area': 'FU_area_Jul16'})

In [ ]:
for df in [Land_2020_5_FU_baseline, Land_2020_10_FU_baseline, Land_2020_20_FU_baseline, Land_2020_50_FU_baseline,\
           Land_2020_100_FU_baseline, Land_2020_200_FU_baseline, Land_2020_500_FU_baseline, Land_2020_1000_FU_baseline]:
    df['FU_area_Jul16'] = df['FU_area_Jul16'].replace(0, 1e-6)

In [ ]:
Land_2020_5_FU_baseline

In [ ]:
# Merge baseline back to original df
Land_2020_5_FU_df = Land_2020_5_FU_df.merge(Land_2020_5_FU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_10_FU_df = Land_2020_10_FU_df.merge(Land_2020_10_FU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_20_FU_df = Land_2020_20_FU_df.merge(Land_2020_20_FU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_50_FU_df = Land_2020_50_FU_df.merge(Land_2020_50_FU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_100_FU_df = Land_2020_100_FU_df.merge(Land_2020_100_FU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_200_FU_df = Land_2020_200_FU_df.merge(Land_2020_200_FU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_500_FU_df = Land_2020_500_FU_df.merge(Land_2020_500_FU_baseline, on=['Land', 'depth_cat'], how='left')
Land_2020_1000_FU_df = Land_2020_1000_FU_df.merge(Land_2020_1000_FU_baseline, on=['Land', 'depth_cat'], how='left')

In [ ]:
for df in [Land_2020_5_FU_df, Land_2020_10_FU_df, Land_2020_20_FU_df, Land_2020_50_FU_df,\
           Land_2020_100_FU_df, Land_2020_200_FU_df, Land_2020_500_FU_df, Land_2020_1000_FU_df]:
    df['FU_area_non_zero'] = df.FU_area.replace(0, 1e-6)
    df['pct_growth'] = (df['FU_area_non_zero'] - df['FU_area_Jul16']) / df['FU_area_Jul16'] * 100

In [ ]:
Land_2020_10_FU_df[Land_2020_10_FU_df.Land == 'Berlin']

### Make an order for plotting

In [ ]:
for df in [Land_2020_5_FU_df, Land_2020_10_FU_df, Land_2020_20_FU_df, Land_2020_50_FU_df,\
           Land_2020_100_FU_df, Land_2020_200_FU_df, Land_2020_500_FU_df, Land_2020_1000_FU_df]:
    df['risk_level'] = df.depth_cat.map(risk_level)

In [ ]:
Land_2020_100_jan25_risky_FU = Land_2020_100_FU_df[(Land_2020_100_FU_df['month_year'] == 'Jan-25') & (Land_2020_100_FU_df['risk_level'] == 'high_risk')]

In [ ]:
Land_2020_100_jan25_risky_FU[Land_2020_100_jan25_risky_FU.Land == 'Brandenburg']

In [ ]:
Land_2020_100_jan25_risky_sum_FU = Land_2020_100_jan25_risky_FU.groupby(['Land', 'risk_level']).sum(['FU_area_non_zero', 'FU_area_Jul16']).reset_index()[['Land', 'risk_level', 'FU_area_non_zero', 'FU_area_Jul16']]

In [ ]:
Land_2020_100_jan25_risky_sum_FU['pct_growth_risky'] = \
 (Land_2020_100_jan25_risky_sum_FU.FU_area_non_zero - Land_2020_100_jan25_risky_sum_FU.FU_area_Jul16) / Land_2020_100_jan25_risky_sum_FU.FU_area_Jul16 * 100

In [ ]:
Land_2020_100_jan25_risky_sum_FU = Land_2020_100_jan25_risky_sum_FU[Land_2020_100_jan25_risky_sum_FU.Land != "Germany"]

In [ ]:
Land_2020_100_jan25_risky_sum_FU.sort_values(by='pct_growth_risky', ascending=False).Land.tolist()

In [ ]:
fu_plot_order = Land_2020_100_jan25_risky_sum_FU.sort_values(by='pct_growth_risky', ascending=False).Land.tolist()

## Pluvial flooding defended

### Data Preparation

In [ ]:
# Select columns that start with 'CU' or are exactly 'Land'
pd_columns = ['Land'] + [col for col in all_columns if col.startswith('PD')]

Land_2020_5_PD = Land_2020_5_raw[pd_columns]
Land_2020_10_PD = Land_2020_10_raw[pd_columns]
Land_2020_20_PD = Land_2020_20_raw[pd_columns]
Land_2020_50_PD = Land_2020_50_raw[pd_columns]
Land_2020_100_PD = Land_2020_100_raw[pd_columns]
Land_2020_200_PD = Land_2020_200_raw[pd_columns]
Land_2020_500_PD = Land_2020_500_raw[pd_columns]
Land_2020_1000_PD = Land_2020_1000_raw[pd_columns]

In [ ]:
Land_2020_5_PD.Land.unique()

In [ ]:
Land_2020_5_PD_long = Land_2020_5_PD.melt(id_vars=['Land'], var_name='month_year_depth', value_name='PD_area')
Land_2020_10_PD_long = Land_2020_10_PD.melt(id_vars=['Land'], var_name='month_year_depth', value_name='PD_area')
Land_2020_20_PD_long = Land_2020_20_PD.melt(id_vars=['Land'], var_name='month_year_depth', value_name='PD_area')
Land_2020_50_PD_long = Land_2020_50_PD.melt(id_vars=['Land'], var_name='month_year_depth', value_name='PD_area')
Land_2020_100_PD_long = Land_2020_100_PD.melt(id_vars=['Land'], var_name='month_year_depth', value_name='PD_area')
Land_2020_200_PD_long = Land_2020_200_PD.melt(id_vars=['Land'], var_name='month_year_depth', value_name='PD_area')
Land_2020_500_PD_long = Land_2020_500_PD.melt(id_vars=['Land'], var_name='month_year_depth', value_name='PD_area')
Land_2020_1000_PD_long = Land_2020_1000_PD.melt(id_vars=['Land'], var_name='month_year_depth', value_name='PD_area')

In [ ]:
Land_2020_5_PD_long.head()

In [ ]:
for df in [Land_2020_5_PD_long, Land_2020_10_PD_long, Land_2020_20_PD_long, Land_2020_50_PD_long,\
           Land_2020_100_PD_long, Land_2020_200_PD_long, Land_2020_500_PD_long, Land_2020_1000_PD_long]:
    # Split the 'month-year-depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(PD)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Optional: drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)

In [ ]:
Land_2020_5_PD_long.head()

In [ ]:
Land_2020_5_PD_settle = Land_2020_5_PD_long.groupby(['Land', 'month_year']).sum('PD_area').reset_index()
Land_2020_10_PD_settle = Land_2020_10_PD_long.groupby(['Land', 'month_year']).sum('PD_area').reset_index()
Land_2020_20_PD_settle = Land_2020_20_PD_long.groupby(['Land', 'month_year']).sum('PD_area').reset_index()
Land_2020_50_PD_settle = Land_2020_50_PD_long.groupby(['Land', 'month_year']).sum('PD_area').reset_index()
Land_2020_100_PD_settle = Land_2020_100_PD_long.groupby(['Land', 'month_year']).sum('PD_area').reset_index()
Land_2020_200_PD_settle = Land_2020_200_PD_long.groupby(['Land', 'month_year']).sum('PD_area').reset_index()
Land_2020_500_PD_settle = Land_2020_500_PD_long.groupby(['Land', 'month_year']).sum('PD_area').reset_index()
Land_2020_1000_PD_settle = Land_2020_1000_PD_long.groupby(['Land', 'month_year']).sum('PD_area').reset_index()

In [ ]:
for df in [Land_2020_5_PD_settle, Land_2020_10_PD_settle, Land_2020_20_PD_settle, Land_2020_50_PD_settle,\
           Land_2020_100_PD_settle, Land_2020_200_PD_settle, Land_2020_500_PD_settle, Land_2020_1000_PD_settle]:
    df.columns = ['Land', 'month_year', 'settle_area']

In [ ]:
Land_2020_5_PD_df = pd.merge(Land_2020_5_PD_long, Land_2020_5_PD_settle, on=['Land', 'month_year'], how='left')
Land_2020_10_PD_df = pd.merge(Land_2020_10_PD_long, Land_2020_10_PD_settle, on=['Land', 'month_year'], how='left')
Land_2020_20_PD_df = pd.merge(Land_2020_20_PD_long, Land_2020_20_PD_settle, on=['Land', 'month_year'], how='left')
Land_2020_50_PD_df = pd.merge(Land_2020_50_PD_long, Land_2020_50_PD_settle, on=['Land', 'month_year'], how='left')
Land_2020_100_PD_df = pd.merge(Land_2020_100_PD_long, Land_2020_100_PD_settle, on=['Land', 'month_year'], how='left')
Land_2020_200_PD_df = pd.merge(Land_2020_200_PD_long, Land_2020_200_PD_settle, on=['Land', 'month_year'], how='left')
Land_2020_500_PD_df = pd.merge(Land_2020_500_PD_long, Land_2020_500_PD_settle, on=['Land', 'month_year'], how='left')
Land_2020_1000_PD_df = pd.merge(Land_2020_1000_PD_long, Land_2020_1000_PD_settle, on=['Land', 'month_year'], how='left')

In [ ]:
Land_2020_5_PD_df.head()

In [ ]:
for df in [Land_2020_5_PD_df, Land_2020_10_PD_df, Land_2020_20_PD_df, Land_2020_50_PD_df,\
           Land_2020_100_PD_df, Land_2020_200_PD_df, Land_2020_500_PD_df, Land_2020_1000_PD_df]:
    df['depth_cat_area_perc'] = (df.PD_area / df.settle_area)*100

In [ ]:
Land_2020_5_PD_df.head(n=20)

In [ ]:
Land_2020_5_PD_baseline = Land_2020_5_PD_df[Land_2020_5_PD_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'PD_area']]
Land_2020_10_PD_baseline = Land_2020_10_PD_df[Land_2020_10_PD_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'PD_area']]
Land_2020_20_PD_baseline = Land_2020_20_PD_df[Land_2020_20_PD_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'PD_area']]
Land_2020_50_PD_baseline = Land_2020_50_PD_df[Land_2020_50_PD_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'PD_area']]
Land_2020_100_PD_baseline = Land_2020_100_PD_df[Land_2020_100_PD_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'PD_area']]
Land_2020_200_PD_baseline = Land_2020_200_PD_df[Land_2020_200_PD_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'PD_area']]
Land_2020_500_PD_baseline = Land_2020_500_PD_df[Land_2020_500_PD_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'PD_area']]
Land_2020_1000_PD_baseline = Land_2020_1000_PD_df[Land_2020_1000_PD_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'PD_area']]

In [ ]:
Land_2020_20_PD_baseline

In [ ]:
Land_2020_5_PD_baseline = Land_2020_5_PD_baseline.rename(columns={'PD_area': 'PD_area_Jul16'})
Land_2020_10_PD_baseline = Land_2020_10_PD_baseline.rename(columns={'PD_area': 'PD_area_Jul16'})
Land_2020_20_PD_baseline = Land_2020_20_PD_baseline.rename(columns ={'PD_area': 'PD_area_Jul16'})
Land_2020_50_PD_baseline = Land_2020_50_PD_baseline.rename(columns={'PD_area': 'PD_area_Jul16'})
Land_2020_100_PD_baseline = Land_2020_100_PD_baseline.rename(columns={'PD_area': 'PD_area_Jul16'})
Land_2020_200_PD_baseline = Land_2020_200_PD_baseline.rename(columns={'PD_area': 'PD_area_Jul16'})
Land_2020_500_PD_baseline = Land_2020_500_PD_baseline.rename(columns={'PD_area': 'PD_area_Jul16'})
Land_2020_1000_PD_baseline = Land_2020_1000_PD_baseline.rename(columns={'PD_area': 'PD_area_Jul16'})

In [ ]:
for df in [Land_2020_5_PD_baseline, Land_2020_10_PD_baseline, Land_2020_20_PD_baseline, Land_2020_50_PD_baseline,
           Land_2020_100_PD_baseline, Land_2020_200_PD_baseline, Land_2020_500_PD_baseline, Land_2020_1000_PD_baseline]:
    df['PD_area_Jul16'] = df['PD_area_Jul16'].replace(0, 1e-6)


In [ ]:
suffixes = [5, 10, 20, 50, 100, 200, 500, 1000]

for suffix in suffixes:
    df_name = f'Land_2020_{suffix}_PD_df'
    baseline_name = f'Land_2020_{suffix}_PD_baseline'

    # Use globals() to access the variables dynamically
    merged_df = globals()[df_name].merge(globals()[baseline_name], on=['Land', 'depth_cat'], how='left')

    # Update the original DataFrame with the merged result
    globals()[df_name] = merged_df

In [ ]:
suffixes = [5, 10, 20, 50, 100, 200, 500, 1000]

for suffix in suffixes:
    df_name = f'Land_2020_{suffix}_PD_df'
    df = globals()[df_name]

    df['PD_area_non_zero'] = df['PD_area'].replace(0, 1e-6)
    df['pct_growth'] = (df['PD_area_non_zero'] - df['PD_area_Jul16']) / df['PD_area_Jul16'] * 100

    globals()[df_name] = df  # Reassign to update the global variable

### Make an order for plotting

In [ ]:
for df in [Land_2020_5_PD_df, Land_2020_10_PD_df, Land_2020_20_PD_df, Land_2020_50_PD_df,\
           Land_2020_100_PD_df, Land_2020_200_PD_df, Land_2020_500_PD_df, Land_2020_1000_PD_df]:
    df['risk_level'] = df.depth_cat.map(risk_level)

In [ ]:
Land_2020_100_jan25_risky_PD = Land_2020_100_PD_df[(Land_2020_100_PD_df['month_year'] == 'Jan-25') & (Land_2020_100_PD_df['risk_level'] == 'high_risk')]

In [ ]:
Land_2020_100_jan25_risky_PD[Land_2020_100_jan25_risky_PD.Land == 'Brandenburg']

In [ ]:
Land_2020_100_jan25_risky_sum_PD = Land_2020_100_jan25_risky_PD.groupby(['Land', 'risk_level']).sum(['PD_area_non_zero', 'PD_area_Jul16']).reset_index()[['Land', 'risk_level', 'PD_area_non_zero', 'PD_area_Jul16']]

In [ ]:
Land_2020_100_jan25_risky_sum_PD['pct_growth_risky'] = \
 (Land_2020_100_jan25_risky_sum_PD.PD_area_non_zero - Land_2020_100_jan25_risky_sum_PD.PD_area_Jul16) / Land_2020_100_jan25_risky_sum_PD.PD_area_Jul16 * 100

In [ ]:
len(Land_2020_100_jan25_risky_sum_PD)

In [ ]:
Land_2020_100_jan25_risky_sum_PD = Land_2020_100_jan25_risky_sum_PD[Land_2020_100_jan25_risky_sum_PD.Land != "Germany"]

In [ ]:
Land_2020_100_jan25_risky_sum_PD.sort_values(by='pct_growth_risky', ascending=False).Land.tolist()

In [ ]:
pd_plot_order = Land_2020_100_jan25_risky_sum_PD.sort_values(by='pct_growth_risky', ascending=False).Land.tolist()

## Combined flooding (max)

### Data Preparation

In [ ]:
# Select columns that start with 'MAX'
max_columns = ['Land'] + [col for col in all_columns if col.startswith('MAX')]

Land_2020_5_MAX = Land_2020_5_raw[max_columns]
Land_2020_10_MAX = Land_2020_10_raw[max_columns]
Land_2020_20_MAX = Land_2020_20_raw[max_columns]
Land_2020_50_MAX = Land_2020_50_raw[max_columns]
Land_2020_100_MAX = Land_2020_100_raw[max_columns]
Land_2020_200_MAX = Land_2020_200_raw[max_columns]
Land_2020_500_MAX = Land_2020_500_raw[max_columns]
Land_2020_1000_MAX = Land_2020_1000_raw[max_columns]

In [ ]:
Land_2020_5_MAX.Land.unique()

In [ ]:
Land_2020_5_MAX_long = Land_2020_5_MAX.melt(id_vars=['Land'], var_name='month_year_depth', value_name='MAX_area')
Land_2020_10_MAX_long = Land_2020_10_MAX.melt(id_vars=['Land'], var_name='month_year_depth', value_name='MAX_area')
Land_2020_20_MAX_long = Land_2020_20_MAX.melt(id_vars=['Land'], var_name='month_year_depth', value_name='MAX_area')
Land_2020_50_MAX_long = Land_2020_50_MAX.melt(id_vars=['Land'], var_name='month_year_depth', value_name='MAX_area')
Land_2020_100_MAX_long = Land_2020_100_MAX.melt(id_vars=['Land'], var_name='month_year_depth', value_name='MAX_area')
Land_2020_200_MAX_long = Land_2020_200_MAX.melt(id_vars=['Land'], var_name='month_year_depth', value_name='MAX_area')
Land_2020_500_MAX_long = Land_2020_500_MAX.melt(id_vars=['Land'], var_name='month_year_depth', value_name='MAX_area')
Land_2020_1000_MAX_long = Land_2020_1000_MAX.melt(id_vars=['Land'], var_name='month_year_depth', value_name='MAX_area')

In [ ]:
Land_2020_5_MAX_long.head()

In [ ]:
for df in [Land_2020_5_MAX_long, Land_2020_10_MAX_long, Land_2020_20_MAX_long, Land_2020_50_MAX_long,
           Land_2020_100_MAX_long, Land_2020_200_MAX_long, Land_2020_500_MAX_long, Land_2020_1000_MAX_long]:
    # Split the 'month-year-depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(MAX)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Optional: drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)

In [ ]:
Land_2020_5_MAX_long.head()

In [ ]:
Land_2020_5_MAX_settle = Land_2020_5_MAX_long.groupby(['Land', 'month_year']).sum('MAX_area').reset_index()
Land_2020_10_MAX_settle = Land_2020_10_MAX_long.groupby(['Land', 'month_year']).sum('MAX_area').reset_index()
Land_2020_20_MAX_settle = Land_2020_20_MAX_long.groupby(['Land', 'month_year']).sum('MAX_area').reset_index()
Land_2020_50_MAX_settle = Land_2020_50_MAX_long.groupby(['Land', 'month_year']).sum('MAX_area').reset_index()
Land_2020_100_MAX_settle = Land_2020_100_MAX_long.groupby(['Land', 'month_year']).sum('MAX_area').reset_index()
Land_2020_200_MAX_settle = Land_2020_200_MAX_long.groupby(['Land', 'month_year']).sum('MAX_area').reset_index()
Land_2020_500_MAX_settle = Land_2020_500_MAX_long.groupby(['Land', 'month_year']).sum('MAX_area').reset_index()
Land_2020_1000_MAX_settle = Land_2020_1000_MAX_long.groupby(['Land', 'month_year']).sum('MAX_area').reset_index()

In [ ]:
for df in [Land_2020_5_MAX_settle, Land_2020_10_MAX_settle, Land_2020_20_MAX_settle, Land_2020_50_MAX_settle,
           Land_2020_100_MAX_settle, Land_2020_200_MAX_settle, Land_2020_500_MAX_settle, Land_2020_1000_MAX_settle]:
    df.columns = ['Land', 'month_year', 'settle_area']

In [ ]:
Land_2020_5_MAX_settle

In [ ]:
Land_2020_5_MAX_df = pd.merge(Land_2020_5_MAX_long, Land_2020_5_MAX_settle, on=['Land', 'month_year'], how='left')
Land_2020_10_MAX_df = pd.merge(Land_2020_10_MAX_long, Land_2020_10_MAX_settle, on=['Land', 'month_year'], how='left')
Land_2020_20_MAX_df = pd.merge(Land_2020_20_MAX_long, Land_2020_20_MAX_settle, on=['Land', 'month_year'], how='left')
Land_2020_50_MAX_df = pd.merge(Land_2020_50_MAX_long, Land_2020_50_MAX_settle, on=['Land', 'month_year'], how='left')
Land_2020_100_MAX_df = pd.merge(Land_2020_100_MAX_long, Land_2020_100_MAX_settle, on=['Land', 'month_year'], how='left')
Land_2020_200_MAX_df = pd.merge(Land_2020_200_MAX_long, Land_2020_200_MAX_settle, on=['Land', 'month_year'], how='left')
Land_2020_500_MAX_df = pd.merge(Land_2020_500_MAX_long, Land_2020_500_MAX_settle, on=['Land', 'month_year'], how='left')
Land_2020_1000_MAX_df = pd.merge(Land_2020_1000_MAX_long, Land_2020_1000_MAX_settle, on=['Land', 'month_year'], how='left')

In [ ]:
Land_2020_5_MAX_df.head()

In [ ]:
for df in [Land_2020_5_MAX_df, Land_2020_10_MAX_df, Land_2020_20_MAX_df, Land_2020_50_MAX_df,
           Land_2020_100_MAX_df, Land_2020_200_MAX_df, Land_2020_500_MAX_df, Land_2020_1000_MAX_df]:
    df['depth_cat_area_perc'] = (df.MAX_area / df.settle_area) * 100

In [ ]:
Land_2020_5_MAX_df.head(n=20)

In [ ]:
Land_2020_5_MAX_baseline = Land_2020_5_MAX_df[Land_2020_5_MAX_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'MAX_area']]
Land_2020_10_MAX_baseline = Land_2020_10_MAX_df[Land_2020_10_MAX_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'MAX_area']]
Land_2020_20_MAX_baseline = Land_2020_20_MAX_df[Land_2020_20_MAX_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'MAX_area']]
Land_2020_50_MAX_baseline = Land_2020_50_MAX_df[Land_2020_50_MAX_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'MAX_area']]
Land_2020_100_MAX_baseline = Land_2020_100_MAX_df[Land_2020_100_MAX_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'MAX_area']]
Land_2020_200_MAX_baseline = Land_2020_200_MAX_df[Land_2020_200_MAX_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'MAX_area']]
Land_2020_500_MAX_baseline = Land_2020_500_MAX_df[Land_2020_500_MAX_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'MAX_area']]
Land_2020_1000_MAX_baseline = Land_2020_1000_MAX_df[Land_2020_1000_MAX_df['month_year'] == 'Jul-16'][['Land', 'depth_cat', 'MAX_area']]

In [ ]:
Land_2020_20_MAX_baseline

In [ ]:
Land_2020_5_MAX_baseline = Land_2020_5_MAX_baseline.rename(columns={'MAX_area': 'MAX_area_Jul16'})
Land_2020_10_MAX_baseline = Land_2020_10_MAX_baseline.rename(columns={'MAX_area': 'MAX_area_Jul16'})
Land_2020_20_MAX_baseline = Land_2020_20_MAX_baseline.rename(columns={'MAX_area': 'MAX_area_Jul16'})
Land_2020_50_MAX_baseline = Land_2020_50_MAX_baseline.rename(columns={'MAX_area': 'MAX_area_Jul16'})
Land_2020_100_MAX_baseline = Land_2020_100_MAX_baseline.rename(columns={'MAX_area': 'MAX_area_Jul16'})
Land_2020_200_MAX_baseline = Land_2020_200_MAX_baseline.rename(columns={'MAX_area': 'MAX_area_Jul16'})
Land_2020_500_MAX_baseline = Land_2020_500_MAX_baseline.rename(columns={'MAX_area': 'MAX_area_Jul16'})
Land_2020_1000_MAX_baseline = Land_2020_1000_MAX_baseline.rename(columns={'MAX_area': 'MAX_area_Jul16'})

In [ ]:
for df in [Land_2020_5_MAX_baseline, Land_2020_10_MAX_baseline, Land_2020_20_MAX_baseline, Land_2020_50_MAX_baseline,
           Land_2020_100_MAX_baseline, Land_2020_200_MAX_baseline, Land_2020_500_MAX_baseline, Land_2020_1000_MAX_baseline]:
    df['MAX_area_Jul16'] = df['MAX_area_Jul16'].replace(0, 1e-6)

In [ ]:
Land_2020_5_MAX_baseline

In [ ]:
suffixes = [5, 10, 20, 50, 100, 200, 500, 1000]

for suffix in suffixes:
    df_name = f'Land_2020_{suffix}_MAX_df'
    baseline_name = f'Land_2020_{suffix}_MAX_baseline'

    # Use globals() to access the variables dynamically
    merged_df = globals()[df_name].merge(globals()[baseline_name], on=['Land', 'depth_cat'], how='left')

    # Update the original DataFrame with the merged result
    globals()[df_name] = merged_df

In [ ]:
Land_2020_5_MAX_df

In [ ]:
suffixes = [5, 10, 20, 50, 100, 200, 500, 1000]

for suffix in suffixes:
    df_name = f'Land_2020_{suffix}_MAX_df'
    df = globals()[df_name]

    df['MAX_area_non_zero'] = df['MAX_area'].replace(0, 1e-6)
    df['pct_growth'] = (df['MAX_area_non_zero'] - df['MAX_area_Jul16']) / df['MAX_area_Jul16'] * 100

    globals()[df_name] = df  # Reassign to update the global variable

### Make an order for plotting

In [ ]:
for df in [Land_2020_5_MAX_df, Land_2020_10_MAX_df, Land_2020_20_MAX_df, Land_2020_50_MAX_df,\
           Land_2020_100_MAX_df, Land_2020_200_MAX_df, Land_2020_500_MAX_df, Land_2020_1000_MAX_df]:
    df['risk_level'] = df.depth_cat.map(risk_level)

In [ ]:
Land_2020_100_jan25_risky_MAX = Land_2020_100_MAX_df[(Land_2020_100_MAX_df['month_year'] == 'Jan-25') & (Land_2020_100_MAX_df['risk_level'] == 'high_risk')]

In [ ]:
Land_2020_100_jan25_risky_MAX[Land_2020_100_jan25_risky_MAX.Land == 'Brandenburg']

In [ ]:
Land_2020_100_jan25_risky_sum_MAX = Land_2020_100_jan25_risky_MAX.groupby(['Land', 'risk_level']).sum(['MAX_area_non_zero', 'MAX_area_Jul16']).reset_index()[['Land', 'risk_level', 'MAX_area_non_zero', 'MAX_area_Jul16']]

In [ ]:
Land_2020_100_jan25_risky_sum_MAX['pct_growth_risky'] = \
 (Land_2020_100_jan25_risky_sum_MAX.MAX_area_non_zero - Land_2020_100_jan25_risky_sum_MAX.MAX_area_Jul16) / Land_2020_100_jan25_risky_sum_MAX.MAX_area_Jul16 * 100

In [ ]:
Land_2020_100_jan25_risky_sum_MAX['area_growth_risky'] = \
 Land_2020_100_jan25_risky_sum_MAX.MAX_area_non_zero - Land_2020_100_jan25_risky_sum_MAX.MAX_area_Jul16

In [ ]:
Land_2020_100_jan25_risky_sum_MAX = Land_2020_100_jan25_risky_sum_MAX[Land_2020_100_jan25_risky_sum_MAX.Land != "Germany"]

In [ ]:
Land_2020_100_jan25_risky_sum_MAX.sort_values(by='pct_growth_risky', ascending=False).Land.tolist()

In [ ]:
Land_2020_100_jan25_risky_sum_MAX.sort_values(by='area_growth_risky', ascending=False).Land.tolist()

In [ ]:
max_plot_order = Land_2020_100_jan25_risky_sum_MAX.sort_values(by='pct_growth_risky', ascending=False).Land.tolist()

In [ ]:
max_table_order = Land_2020_100_jan25_risky_sum_MAX.sort_values(by='area_growth_risky', ascending=False).Land.tolist()

In [ ]:
Land_2020_100_MAX_df

In [ ]:
# Get the data for July 2016 and January 2025
max_tbl_df = Land_2020_100_MAX_df[(Land_2020_100_MAX_df['month_year']=='Jan-25') & (Land_2020_100_MAX_df['Land'].isin(max_table_order))]

# Calculate the difference
max_tbl_df['area_growth'] = max_tbl_df['MAX_area'] - max_tbl_df['MAX_area_Jul16']

# Sort according to max_table_order
max_tbl_df['Land'] = pd.Categorical(max_tbl_df['Land'], categories=max_table_order, ordered=True)

# Depth category renaming
depth_rename = {
    'P0': '0 m',
    'lt0.15': '< 0.15 m',
    'lt0.5': '< 0.5 m',
    'lt1.5': '< 1.5 m',
    'gt1.5': '> 1.5 m'
}

# Create a new column with readable depth names
max_tbl_df['depth_cat_name'] = max_tbl_df['depth_cat'].map(depth_rename)

# Set the categorical order
depth_order = [depth_rename[k] for k in ['P0', 'lt0.15', 'lt0.5', 'lt1.5', 'gt1.5']]
max_tbl_df['depth_cat_name'] = pd.Categorical(max_tbl_df['depth_cat_name'], categories=depth_order, ordered=True)

# Sort by Land and the new readable depth category
max_tbl_df = max_tbl_df.sort_values(['Land', 'depth_cat_name'])


In [ ]:
max_tbl_df[['Land', 'depth_cat_name', 'MAX_area_Jul16', 'MAX_area', 'area_growth']].reset_index(drop=True).to_csv('max_table_data.csv', index=False)

## Combine individual datasets together for plotting

### data preparation for plot

In [ ]:
dfs_100 = [
    Land_2020_100_MAX_df, Land_2020_100_CU_df, Land_2020_100_FU_df, Land_2020_100_PD_df
]


# Create labels for each scenario
floodings = ['max', 'cu', 'fu','pd']

In [ ]:
# Add scenario labels and combine into one DataFrame
for df, flooding in zip(dfs_100, floodings):
    df['flooding_types'] = flooding

df_all_100 = pd.concat(dfs_100, ignore_index=True)

# Optional: sort month_year for clean x-axis
df_all_100['month_year'] = pd.Categorical(
    df_all_100['month_year'],
    ordered=True,
    categories=sorted(df_all_100['month_year'].unique(), key=lambda x: pd.to_datetime('01-' + x, format='%d-%b-%y'))
)

In [ ]:
# Prepare sorted lists
lands = sorted(df_all_100['Land'].unique())
depth_cats = df_all_100['depth_cat'].unique()

red_shades = [
    '#d3d3d3',  # lightest grey
    # '#c6c6c6',  # light grey
    '#b3b3b3', # medium grey
    # '#999999', #medium grey2
    # '#ffcccc',  # lightest red
    '#ff9999',
    # '#ff6666',
    # '#ff3333',
    '#ff1a1a',
    # '#e60000',
    '#b30000',
    '#800000'   # darkest red
]
depth_colors = dict(zip(depth_cats, red_shades))

In [ ]:
df_filtered = df_all_100[df_all_100['Land'] != "Germany"]
lands_filtered = sorted(df_filtered['Land'].unique())

In [ ]:
df_filtered.columns

### final versions

In [ ]:
flooding_names = {
    'cu': 'Coastal Undefended',
    'fu': 'Fluvial Undefended',
    'pd': 'Pluvial Defended',
    'max': 'MAX'
}

# Define the land orders for each flooding type
flooding_orders = {
    'cu': cu_plot_order,
    'fu': fu_plot_order,
    'pd': pd_plot_order,
    'max': max_plot_order
}

# Depth category renaming
depth_rename = {
    'P0': '0 m',
    'lt0.15': '< 0.15 m',
    'lt0.5': '< 0.5 m',
    'lt1.5': '< 1.5 m',
    'gt1.5': '> 1.5 m'
}

# Only run for max and fluvial
for flooding in ['fu', 'max']:
    land_order = flooding_orders[flooding]  # Get the correct order

    # Compute global y-axis limits for this flooding type
    subset_all = df_filtered[df_filtered['flooding_types'] == flooding]
    y_min = 0
    y_max = np.ceil(subset_all['pct_growth'].max() / 10) * 10

    fig, axs = plt.subplots(4, 4, figsize=(22, 18))
    axs = axs.flatten()

    for idx, land in enumerate(land_order):
        ax = axs[idx]
        subset = subset_all[subset_all['Land'] == land]

        sns.lineplot(
            data=subset,
            x='month_year',
            y='pct_growth',
            hue='depth_cat',
            palette=depth_colors,
            hue_order=depth_cats,
            marker='o',
            ax=ax,
            legend=False
        )

        ax.set_title(f'{land}', fontsize=16)
        ax.set_ylabel('Growth (%)', fontsize=16)
        ax.set_xlabel('', fontsize=16)
        ax.tick_params(axis='x', rotation=45, labelsize=12)
        ax.tick_params(axis='y', labelsize=12)
        ax.grid(True)
        ax.set_ylim(y_min, y_max)

    # Remove extra subplots if fewer than 16
    for j in range(len(land_order), 16):
        fig.delaxes(axs[j])

    # Tight layout before legend
    plt.tight_layout(rect=[0, 0, 1, 0.92])

    # Dummy plot to extract legend handles/labels
    fig_tmp, ax_tmp = plt.subplots()
    dummy = sns.lineplot(
        data=subset_all,
        x='month_year',
        y='pct_growth',
        hue='depth_cat',
        palette=depth_colors,
        hue_order=depth_cats,
        marker='o',
        ax=ax_tmp
    )
    handles, labels = ax_tmp.get_legend_handles_labels()
    plt.close(fig_tmp)

    # Rename labels using depth_rename dict
    labels = [depth_rename.get(l, l) for l in labels]

    # Single legend at bottom center
    fig.legend(
        handles,
        labels,
        title='Exposure level',
        loc='lower center',
        bbox_to_anchor=(0.5, -0.06),
        ncol=len(depth_cats),
        fontsize=14,
        title_fontsize=16
    )


    # # Save without suptitle
    # plt.savefig(
    #     f'/content/drive/MyDrive/Germany_Flood_Study/Land_100yr/final_adjustments/{flooding}_pct_growth_4x4plot.png',
    #     dpi=300,
    #     bbox_inches='tight'
    # )
    # plt.close(fig)


In [ ]:
fig = plt.figure(figsize=(18, 14))

# Define consistent width/height
plot_width = 0.25
plot_height = 0.25

# Define layout for 3 top (symmetric) and 2 bottom
top_axes = [
    fig.add_axes([0.10, 0.42, plot_width, plot_height]),  # Mecklenburg-Vorpommern
    fig.add_axes([0.39, 0.42, plot_width, plot_height]),  # Schleswig-Holstein
    fig.add_axes([0.68, 0.42, plot_width, plot_height])   # Niedersachsen
]
bottom_axes = [
    fig.add_axes([0.22, 0.08, plot_width, plot_height]),  # Bremen
    fig.add_axes([0.56, 0.08, plot_width, plot_height])   # Hamburg
]
axs = top_axes + bottom_axes

# New land order
land_order = [
    'Mecklenburg-Vorpommern',
    'Schleswig-Holstein',
    'Niedersachsen',
    'Bremen',
    'Hamburg'
]

# Subset data
subset_all = df_filtered[df_filtered['flooding_types'] == 'cu']

# Calculate shared max from all except Mecklenburg-Vorpommern
shared_lands = [l for l in land_order if l != 'Mecklenburg-Vorpommern']
shared_max = np.ceil(subset_all[subset_all['Land'].isin(shared_lands)]['pct_growth'].max() / 10) * 10

# Plotting
for idx, land in enumerate(land_order):
    ax = axs[idx]
    subset = subset_all[subset_all['Land'] == land]

    # Unique y-limit for Mecklenburg-Vorpommern
    if land == 'Mecklenburg-Vorpommern':
        y_max = np.ceil(subset['pct_growth'].max() / 10) * 10
    else:
        y_max = shared_max

    sns.lineplot(
        data=subset,
        x='month_year',
        y='pct_growth',
        hue='depth_cat',
        palette=depth_colors,
        hue_order=depth_cats,
        marker='o',
        ax=ax,
        legend=False
    )
    ax.set_title(land, fontsize=16)
    ax.set_ylabel('Growth (%)', fontsize=16)
    ax.set_xlabel('', fontsize=16)
    ax.tick_params(axis='x', rotation=45, labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    ax.grid(True)
    ax.set_ylim(0, y_max)

# Rename legend labels
fig_tmp, ax_tmp = plt.subplots()
dummy = sns.lineplot(
    data=subset_all,
    x='month_year',
    y='pct_growth',
    hue='depth_cat',
    palette=depth_colors,
    hue_order=depth_cats,
    marker='o',
    ax=ax_tmp
)
handles, labels = ax_tmp.get_legend_handles_labels()
plt.close(fig_tmp)

labels = [depth_rename.get(label, label) for label in labels]

# Add legend at bottom center
fig.legend(
    handles,
    labels,
    title='Exposure level',
    loc='lower center',
    bbox_to_anchor=(0.5, -0.08),
    ncol=len(depth_cats),
    fontsize=14,
    title_fontsize=16
)

# No fig.suptitle here (removed)
plt.tight_layout(rect=[0, 0.05, 1, 1])  # leave space for legend

# # Save
# plt.savefig(
#     '/content/drive/MyDrive/Germany_Flood_Study/Land_100yr/final_adjustments/cu_pct_growth_adjusted_plot2.png',
#     dpi=300,
#     bbox_inches='tight'
# )


In [ ]:
# Depth category renaming
depth_rename = {
    'P0': '0 m',
    'lt0.15': '< 0.15 m',
    'lt0.5': '< 0.5 m',
    'lt1.5': '< 1.5 m',
    'gt1.5': '> 1.5 m'
}

fig = plt.figure(figsize=(22, 18))

# Define consistent width/height
plot_width = 0.2
plot_height = 0.17
n_plots = 4
total_width = 1.0
total_plot_width = n_plots * plot_width
remaining_space = total_width - total_plot_width
gap = remaining_space / (n_plots + 1)

# Generate evenly spaced left positions
second_row_lefts = [gap + i * (plot_width + gap) for i in range(n_plots)]

# Top row (row 0)
top_row = [
    fig.add_axes([0.10, 0.77, plot_width, plot_height]),
    fig.add_axes([0.39, 0.77, plot_width, plot_height]),
    fig.add_axes([0.68, 0.77, plot_width, plot_height])
]

# Second row (row 1)
second_row = [
    fig.add_axes([left, 0.53, plot_width, plot_height])
    for left in second_row_lefts
]

# Third row (row 2)
third_row = [
    fig.add_axes([left, 0.29, plot_width, plot_height])
    for left in second_row_lefts
]

# Bottom row (row 3)
bottom_row = [
    fig.add_axes([0.10, 0.05, plot_width, plot_height]),
    fig.add_axes([0.39, 0.05, plot_width, plot_height]),
    fig.add_axes([0.68, 0.05, plot_width, plot_height])
]

# Combine all axes into a single list
axs = top_row + second_row + third_row + bottom_row

# Subset data
subset_all = df_filtered[df_filtered['flooding_types'] == 'pd']

# Reorder Bremen and Hamburg to end
lands_order = [l for l in pd_plot_order if l not in ['Bremen', 'Hamburg']]

# Shared y-axis for first 14
subset_main = subset_all[subset_all['Land'].isin(lands_order)]
shared_y_max = np.ceil(subset_main['pct_growth'].max() / 10) * 10

for idx, land in enumerate(lands_order):
    ax = axs[idx]
    subset = subset_all[subset_all['Land'] == land]

    sns.lineplot(
        data=subset,
        x='month_year',
        y='pct_growth',
        hue='depth_cat',
        palette=depth_colors,
        hue_order=depth_cats,
        marker='o',
        ax=ax,
        legend=False
    )

    ax.set_title(land, fontsize=16)
    ax.set_ylabel('Growth (%)', fontsize=16)
    ax.set_xlabel('', fontsize=16)
    ax.tick_params(axis='x', rotation=45, labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    ax.grid(True)
    ax.set_ylim(0, shared_y_max)

# Use dummy plot for consistent legend
fig_tmp, ax_tmp = plt.subplots()
dummy = sns.lineplot(
    data=subset_all,
    x='month_year',
    y='pct_growth',
    hue='depth_cat',
    palette=depth_colors,
    hue_order=depth_cats,
    marker='o',
    ax=ax_tmp
)
handles, labels = ax_tmp.get_legend_handles_labels()
plt.close(fig_tmp)

labels = [depth_rename.get(label, label) for label in labels]

# Single legend at bottom center with adjusted font size
fig.legend(
    handles,
    labels,
    title='Exposure level',
    loc='lower center',
    bbox_to_anchor=(0.5, -0.06),
    ncol=len(depth_cats),
    fontsize=14,
    title_fontsize=16
)

plt.tight_layout()

# # Save
# plt.savefig(
#     '/content/drive/MyDrive/Germany_Flood_Study/Land_100yr/final_adjustments/pd_pct_growth_adjusted_plot2.png',
#     dpi=300,
#     bbox_inches='tight'
# )


### national level

In [ ]:
df_germany = df_all_100[df_all_100['Land'] == 'Germany']
df_germany.head()

In [ ]:
df_germany_Jan25 = df_germany[df_germany['month_year'] == 'Jan-25']
df_germany_Jan25.columns

In [ ]:
def agg_diff_and_pct(df, prefix):
    """Just clean and rename columns; no aggregation needed."""
    base_col, new_col = f"{prefix}_area_Jul16", f"{prefix}_area_non_zero"

    # Select only relevant columns and drop NaNs for this flooding type
    cols = ["depth_cat", base_col, new_col, "pct_growth"]
    g = df[cols].dropna(subset=[base_col, new_col, "pct_growth"], how="all").copy()

    # Compute diff
    g["diff"] = g[new_col] - g[base_col]

    # Sort and relabel
    g["depth_cat"] = pd.Categorical(g["depth_cat"], categories=depth_order, ordered=True)
    g = g.sort_values("depth_cat")
    g["depth_label"] = g["depth_cat"].map(depth_rename)
    return g


In [ ]:
def _break_marks(ax_top, ax_bot, d=0.015, color="k", lw=1):
    """Draw diagonal marks indicating a broken axis."""
    kwt = dict(transform=ax_top.transAxes, color=color, lw=lw, clip_on=False)
    ax_top.plot((-d, +d), (-d, +d), **kwt)
    ax_top.plot((1 - d, 1 + d), (-d, +d), **kwt)
    kwb = dict(transform=ax_bot.transAxes, color=color, lw=lw, clip_on=False)
    ax_bot.plot((-d, +d), (1 - d, 1 + d), **kwb)
    ax_bot.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwb)

In [ ]:
plt.style.use("default")
def plot_flood_type_manual(
    df,
    ftype,                      # "pd", "fu", "cu", or "max"
    land="Germany",
    bottom_ylim=(-10, 120),
    top_ylim=(900, 1100),
    pct_ylim=(0, 12),
    bottom_ticks=None,          # ← set per-call (e.g., [0,25,50,75,100,125])
    top_ticks=None,             # ← set per-call (e.g., [900,950,1000,1050,1100])
    pct_ticks=None,             # ← set per-call (e.g., [0,2,4,6,8,10,12])
    remove_seam_overlap=True,   # hide duplicated tick exactly at the break
    break_mark_d=0.015,
    figsize=(12, 4.8),
    xtick_rotation=90
):
    """Manual broken-axis plot for a single flooding type with per-call tick control."""
    flood_types = ["max", "cu", "fu", "pd"]
    flooding_names = {'max': 'Combined flood', 'cu': 'Coastal flood',
                      'fu': 'Fluvial flood', 'pd': 'Pluvial flood'}
    depth_order = ['P0', 'lt0.15', 'lt0.5', 'lt1.5', 'gt1.5']
    depth_rename = {'P0':'0 m','lt0.15':'< 0.15 m','lt0.5':'< 0.5 m',
                    'lt1.5':'< 1.5 m','gt1.5':'> 1.5 m'}
    red_shades = ['#d3d3d3','#b3b3b3','#ff9999','#ff1a1a','#b30000','#800000']

    # --- subset & prep ---
    df_land = df[df["Land"].eq(land)].copy()
    subset = df_land[df_land["flooding_types"] == ftype].copy()
    if subset.empty:
        raise ValueError(f"No rows for land={land!r} and flooding_types={ftype!r}.")

    area_new  = f"{ftype.upper()}_area_non_zero"
    area_base = f"{ftype.upper()}_area_Jul16"
    subset["diff"] = subset[area_new] - subset[area_base]

    subset["depth_cat"] = pd.Categorical(subset["depth_cat"],
                                         categories=depth_order, ordered=True)
    subset = subset.sort_values("depth_cat")
    labels = subset["depth_cat"].map(depth_rename).to_numpy()

    colors = red_shades[:len(subset)]
    x = np.arange(len(subset))

    # --- figure layout ---
    fig = plt.figure(figsize=figsize)
    outer = GridSpec(1, 2, figure=fig, wspace=0.35)
    # a little gap helps prevent label collision at the seam
    left_spec = outer[0, 0].subgridspec(2, 1, height_ratios=[1, 1], hspace=0.20)
    ax_top = fig.add_subplot(left_spec[0, 0])
    ax_bot = fig.add_subplot(left_spec[1, 0], sharex=ax_top)
    ax_pct = fig.add_subplot(outer[0, 1])

    # --- left bars on BOTH axes (broken axis) ---
    for ax in (ax_bot, ax_top):
        ax.bar(x, subset["diff"], color=colors, width=0.7,
               edgecolor="black", linewidth=0.6)
        ax.axhline(0, color="0.25", lw=0.8)
        ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))
        ax.set_ylabel("")
    # Keep grid on ONE axis to avoid dotted seam
    ax_top.grid(True, axis="y", ls=":", alpha=0.4)
    ax_bot.grid(False, axis="y")

    # --- ticks & labels once ---
    ax_top.set_xticks(x);  ax_bot.set_xticks(x)
    ax_top.set_xticklabels([])  # hide on top
    ax_bot.set_xticklabels(labels, rotation=xtick_rotation, ha="right")
    ax_top.tick_params(axis="x", labelbottom=False)
    ax_bot.tick_params(axis="x", labelbottom=True, bottom=True)

    # --- manual limits ---
    ax_bot.set_ylim(*bottom_ylim)
    ax_top.set_ylim(*top_ylim)
    ax_pct.set_ylim(*pct_ylim)

    # --- per-call tick breaks ---
    if bottom_ticks is not None:
        ax_bot.set_yticks(bottom_ticks)
    if top_ticks is not None:
        ax_top.set_yticks(top_ticks)
    if pct_ticks is not None:
        ax_pct.set_yticks(pct_ticks)

    # remove duplicated tick right at the break (optional)
    if remove_seam_overlap and (bottom_ticks is not None) and (top_ticks is not None):
        if len(bottom_ticks) and len(top_ticks) and bottom_ticks[-1] == top_ticks[0]:
            ax_bot.set_yticks(bottom_ticks[:-1])

    # --- break styling ---
    ax_top.spines["bottom"].set_visible(False)
    ax_bot.spines["top"].set_visible(False)
    _break_marks(ax_top, ax_bot, d=break_mark_d)

    # --- right panel: % growth ---
    ax_pct.bar(x, subset["pct_growth"], color=colors, width=0.7,
               edgecolor="black", linewidth=0.6)
    ax_pct.axhline(0, color="0.25", lw=0.8)
    ax_pct.grid(True, axis="y", ls=":", alpha=0.4)
    ax_pct.set_xticks(x)
    ax_pct.set_xticklabels(labels, rotation=xtick_rotation, ha="right")
    ax_pct.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))
    ax_pct.yaxis.tick_right()
    ax_pct.yaxis.set_label_position("right")

    # --- titles & shared labels ---
    fig.suptitle(flooding_names.get(ftype, ftype.upper()),
                 y=0.99, fontsize=14, fontweight="semibold")
    fig.text(0.06, 0.5, r"Exposure increase (km$^2$)",
             rotation=90, va="center", ha="center", fontsize=12)
    fig.text(0.96, 0.5, "Growth (%)",
             rotation=270, va="center", ha="center", fontsize=12)

    fig.tight_layout(rect=[0.10, 0.10, 0.93, 0.96])
    return fig


In [ ]:
diff_limits = {
    "max": {"bottom": (0, 300), "top": (900, 1000)},
    "cu":  {"bottom": (0, 25), "top": (900, 1200)},
    "fu":  {"bottom": (0, 45), "top": (900, 1100)},
    "pd":  {"bottom": (0, 120), "top": (900, 1100)},
}
pct_limits = {"max": (0, 12), "cu": (0, 16), "fu": (0, 10), "pd": (0, 12)}

In [ ]:
fig = plot_flood_type_manual(
    df_germany_Jan25, "max", land="Germany",
    bottom_ylim=(0, 125), top_ylim=(900, 1000), pct_ylim=(0, 12),
    bottom_ticks=[0, 20, 40, 60, 80, 100],
    top_ticks=[900, 950, 1000],
    pct_ticks=[0, 3, 6, 9, 12],
    break_mark_d=0.02
)


In [ ]:
def plot_flood_types_stack(
    df,
    types,                    # e.g., ["cu", "fu", "pd"]
    land="Germany",
    ylims=None,               # {"cu": ((0,40),(900,1100)), ...}
    pct_ylims=None,           # {"cu": (0,12), ...}
    yticks=None,              # {"cu": ([0,10,20],[900,1000]), ...}
    pct_ticks=None,           # {"cu": [0,2,4,6,8,10,12], ...}
    figsize=(12, 4.8),
    break_mark_d=0.015,
    xtick_rotation=90
):
    from matplotlib.gridspec import GridSpec
    import matplotlib.ticker as mticker
    import matplotlib.pyplot as plt
    import numpy as np

    flood_types = ["max", "cu", "fu", "pd"]
    flooding_names = {'max': 'Combined flood', 'cu': 'Coastal flood',
                      'fu': 'Fluvial flood', 'pd': 'Pluvial flood'}
    depth_order = ['P0', 'lt0.15', 'lt0.5', 'lt1.5', 'gt1.5']
    depth_rename = {'P0':'0 m','lt0.15':'< 0.15 m','lt0.5':'< 0.5 m',
                    'lt1.5':'< 1.5 m','gt1.5':'> 1.5 m'}
    red_shades = ['#d3d3d3','#b3b3b3','#ff9999','#ff1a1a','#b30000','#800000']

    n = len(types)
    fig = plt.figure(figsize=(figsize[0], figsize[1] * n))
    outer = GridSpec(n, 2, figure=fig, hspace=0.6, wspace=0.35)

    for i, ftype in enumerate(types):
        # Left column (broken axis)
        left_spec = outer[i, 0].subgridspec(2, 1, height_ratios=[1, 1], hspace=0.15)
        ax_top = fig.add_subplot(left_spec[0, 0])
        ax_bot = fig.add_subplot(left_spec[1, 0], sharex=ax_top)
        ax_pct = fig.add_subplot(outer[i, 1])

        # ---- data ----
        df_land = df[df["Land"].eq(land)].copy()
        subset = df_land[df_land["flooding_types"] == ftype].copy()
        if subset.empty:
            for ax in (ax_top, ax_bot, ax_pct): ax.axis("off")
            continue

        area_new  = f"{ftype.upper()}_area_non_zero"
        area_base = f"{ftype.upper()}_area_Jul16"
        subset["diff"] = subset[area_new] - subset[area_base]
        subset["depth_cat"] = pd.Categorical(subset["depth_cat"], categories=depth_order, ordered=True)
        subset = subset.sort_values("depth_cat")
        labels = subset["depth_cat"].map(depth_rename).to_numpy()
        colors = red_shades[:len(subset)]
        x = np.arange(len(subset))

        # ---- left bars (both panels) ----
        for ax in (ax_bot, ax_top):
            ax.bar(x, subset["diff"], color=colors, width=0.7,
                   edgecolor="black", linewidth=0.6)
            ax.axhline(0, color="0.25", lw=0.8)
            ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))
        # turn OFF grids everywhere
        ax_top.grid(False); ax_bot.grid(False); ax_pct.grid(False)

        # ---- x ticks & labels ----
        ax_top.set_xticks(x);  ax_bot.set_xticks(x)
        # hard-off on top so it won't reappear at the break
        ax_top.set_xticklabels([])
        ax_top.tick_params(axis="x", which="both",
                           bottom=False, top=False,
                           labelbottom=False, labeltop=False)
        # force-visible on bottom
        ax_bot.set_xticklabels(labels, rotation=xtick_rotation, ha="right")
        ax_bot.tick_params(axis="x", which="both",
                           bottom=True, labelbottom=True, top=False, labeltop=False)
        plt.setp(ax_bot.get_xticklabels(), visible=True)
        plt.setp(ax_top.get_xticklabels(), visible=False)

        # ---- y-lims and ticks ----
        bot_ylim, top_ylim = (ylims.get(ftype) if ylims else ((0, 100), (900, 1100)))
        ax_bot.set_ylim(*bot_ylim)
        ax_top.set_ylim(*top_ylim)
        ax_pct.set_ylim(*(pct_ylims.get(ftype) if pct_ylims else (0, 12)))

        if yticks and ftype in yticks:
            bt, tt = yticks[ftype]
            ax_bot.set_yticks(bt)
            ax_top.set_yticks(tt)
            # drop duplicate seam tick if present
            if len(bt) and len(tt) and bt[-1] == tt[0]:
                ax_bot.set_yticks(bt[:-1])

        if pct_ticks and ftype in pct_ticks:
            ax_pct.set_yticks(pct_ticks[ftype])
        ax_pct.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))

        # ---- break marks ----
        ax_top.spines["bottom"].set_visible(False)
        ax_bot.spines["top"].set_visible(False)
        _break_marks(ax_top, ax_bot, d=break_mark_d)

        # ---- right panel bars ----
        ax_pct.bar(x, subset["pct_growth"], color=colors, width=0.7,
                   edgecolor="black", linewidth=0.6)
        ax_pct.axhline(0, color="0.25", lw=0.8)
        ax_pct.set_xticks(x)
        ax_pct.set_xticklabels(labels, rotation=xtick_rotation, ha="right")
        ax_pct.yaxis.tick_right()
        ax_pct.yaxis.set_label_position("right")

        # ---- row title ----
        posL, posR = ax_bot.get_position(), ax_pct.get_position()
        x_mid = (posL.x0 + posR.x1) / 2
        y_row = max(posL.y1, posR.y1) + 0.02
        fig.text(x_mid, y_row, flooding_names.get(ftype, ftype.upper()),
                 ha="center", va="bottom", fontsize=13, fontweight="semibold")

    # ---- shared y-labels ----
    fig.text(0.06, 0.5, r"Exposure increase (km$^2$)", rotation=90,
             va="center", ha="center", fontsize=12)
    fig.text(0.95, 0.5, "Growth (%)", rotation=270,
             va="center", ha="center", fontsize=12)

    plt.tight_layout(rect=[0.10, 0.05, 0.93, 0.97])
    return fig


In [ ]:
fig = plot_flood_types_stack(
    df_germany_Jan25,
    types=["cu", "fu", "pd"],
    land="Germany",
    ylims={
        "cu": ((0, 18), (1000, 1200)),
        "fu": ((0, 42), (900, 1120)),
        "pd": ((0, 125), (900, 1150))
    },
    yticks={
        "cu": ([0,5,10,15], [1000,1200]),
        "fu": ([0,10,20,30,40], [900,1000,1100]),
        "pd": ([0,25,50,75,100], [900,1000,1100])
    },
    pct_ylims={"cu": (0,16), "fu": (0,12), "pd": (0,12)},
    pct_ticks={"cu": [0,3,5,10,15],
               "fu": [0,3,6,9,12],
               "pd": [0,3,6,9,12]},
    break_mark_d=0.02
)


In [ ]:
# Set up plot titles
flooding_names = {
    'cu': 'Coastal flood',
    'fu': 'Fluvial flood',
    'pd': 'Pluvial flood',
    'max': 'Combined flood'
}

# Rename depth categories
depth_rename = {
    'P0': '0 m',
    'lt0.15': '< 0.15 m',
    'lt0.5': '< 0.5 m',
    'lt1.5': '< 1.5 m',
    'gt1.5': '> 1.5 m'
}

# Create 2 rows x 2 columns
fig, axs = plt.subplots(2, 2, figsize=(16, 12), sharey=True)
axs = axs.flatten()

legend_handles, legend_labels = None, None  # store legend info

for idx, flooding in enumerate(['cu', 'fu', 'pd', 'max']):
    ax = axs[idx]

    # Subset for whole Germany & specific flooding type
    subset = df_germany[df_germany['flooding_types'] == flooding]

    # Plot
    sns.lineplot(
        data=subset,
        x='month_year',
        y='pct_growth',
        hue='depth_cat',
        palette=depth_colors,
        hue_order=depth_cats,
        marker='o',
        ax=ax
    )

    # Format
    ax.set_title(flooding_names[flooding], fontsize=16)
    ax.set_xlabel('', fontsize=14)
    if idx % 2 == 0:
        ax.set_ylabel('Growth (%)', fontsize=14)
    else:
        ax.set_ylabel('')
    ax.tick_params(axis='x', rotation=45, labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    ax.grid(True)

    # Capture legend handles & labels from the first subplot only
    if idx == 0:
        legend_handles, legend_labels = ax.get_legend_handles_labels()
        legend_labels = [depth_rename.get(label, label) for label in legend_labels]

    ax.get_legend().remove()  # remove subplot legends

# Add single legend at bottom center
fig.legend(
    legend_handles,
    legend_labels,
    title='Exposure level',
    loc='lower center',
    bbox_to_anchor=(0.5, -0.02),
    ncol=len(depth_rename),
    fontsize=12,
    title_fontsize=13
)

plt.tight_layout(rect=[0, 0.05, 1, 1])  # make space for legend

# Save
plt.savefig(
    '/content/drive/MyDrive/Germany_Flood_Study/Land_100yr/final_adjustments/Germany_pct_growth_plot.png',
    dpi=300, bbox_inches='tight'
)
plt.show()
